# Gweizy Model Training Notebook

Train all gas prediction models for Gweizy.

## Instructions:
1. Upload your `gas_data.db` file (from `backend/gas_data.db`)
2. Run all cells
3. Download the trained models zip file
4. Extract to `backend/models/saved_models/` and push to GitHub

In [ ]:
# Install dependencies
!pip install -q scikit-learn pandas numpy joblib lightgbm xgboost matplotlib seaborn optuna

In [ ]:
# Upload your gas_data.db file
from google.colab import files
import os

print("Upload your gas_data.db file from backend/gas_data.db")
uploaded = files.upload()

if 'gas_data.db' in uploaded:
    print(f"\n✅ Uploaded gas_data.db ({len(uploaded['gas_data.db']) / 1024 / 1024:.1f} MB)")
else:
    print("❌ Please upload gas_data.db")

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load data from database
conn = sqlite3.connect('gas_data.db')
df = pd.read_sql("""
    SELECT timestamp, current_gas as gas, base_fee, priority_fee, 
           block_number, gas_used, gas_limit, utilization
    FROM gas_prices ORDER BY timestamp ASC
""", conn)
conn.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp').sort_index()

print(f"Total records: {len(df):,}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

# === IMPROVED: Resample to 30-second intervals (was 1-min, losing too much data) ===
print("\nResampling to 30-second intervals (preserves more data)...")
df = df.resample('30s').mean().dropna(subset=['gas'])
print(f"After resample: {len(df):,} records")

# Find segments (gap > 30 min = new segment)
df['time_diff'] = df.index.to_series().diff()
df['segment'] = (df['time_diff'] > pd.Timedelta(minutes=30)).cumsum()

segment_sizes = df.groupby('segment').size()
print(f"\nSegments found: {len(segment_sizes)}")
print(f"Segment sizes: {segment_sizes.sort_values(ascending=False).head(10).tolist()}")

# === IMPROVED: Lower threshold from 120 to 30 minutes (keeps more segments) ===
MIN_SEGMENT_SIZE = 60  # 30 minutes at 30-sec intervals = 60 records
good_segments = segment_sizes[segment_sizes >= MIN_SEGMENT_SIZE].index.tolist()
df = df[df['segment'].isin(good_segments)]
print(f"\nKeeping {len(good_segments)} segments with >= 30 minutes of data")
print(f"Total usable records: {len(df):,}")

# === DATA SUFFICIENCY CHECK ===
MIN_REQUIRED_SAMPLES = 10000
if len(df) < MIN_REQUIRED_SAMPLES:
    print(f"\n⚠️  WARNING: Only {len(df):,} samples. Recommend at least {MIN_REQUIRED_SAMPLES:,}")
    print("   Models may underperform. Consider collecting more data.")
else:
    print(f"\n✓ Data sufficiency check passed: {len(df):,} samples")

RECORDS_PER_HOUR = 120  # 30-sec intervals = 120 records per hour

In [ ]:
# Fetch ETH Price Data (External Feature)
import requests

print("="*60)
print("FETCHING EXTERNAL DATA: ETH PRICE")
print("="*60)

def fetch_eth_price_history(start_date, end_date):
    """Fetch ETH price history from CoinGecko API (free, no key needed)"""
    try:
        start_ts = int(start_date.timestamp())
        end_ts = int(end_date.timestamp())
        
        url = f"https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range"
        params = {
            'vs_currency': 'usd',
            'from': start_ts,
            'to': end_ts
        }
        
        print(f"Fetching ETH prices from {start_date} to {end_date}...")
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            prices = data.get('prices', [])
            
            eth_df = pd.DataFrame(prices, columns=['timestamp', 'eth_price'])
            eth_df['timestamp'] = pd.to_datetime(eth_df['timestamp'], unit='ms')
            eth_df = eth_df.set_index('timestamp')
            
            print(f"  Fetched {len(eth_df)} ETH price points")
            return eth_df
        else:
            print(f"  API returned status {response.status_code}")
            return None
            
    except Exception as e:
        print(f"  Failed to fetch ETH prices: {e}")
        return None

# Fetch ETH data for our date range
eth_data = fetch_eth_price_history(df.index.min(), df.index.max())

has_eth_data = False
if eth_data is not None and len(eth_data) > 0:
    # Resample to 30-second intervals to match our gas data
    eth_data = eth_data.resample('30s').ffill()
    
    # Merge with gas data
    df = df.join(eth_data, how='left')
    df['eth_price'] = df['eth_price'].ffill().bfill()
    
    eth_coverage = df['eth_price'].notna().mean()
    print(f"  ETH price coverage: {eth_coverage:.1%}")
    
    if eth_coverage > 0.5:
        has_eth_data = True
        print("  ✓ ETH price data integrated")
    else:
        print("  ⚠️ Low ETH coverage, features may be limited")
else:
    print("  ⚠️ No ETH price data available - training without ETH features")
    df['eth_price'] = np.nan

HAS_ETH_PRICE = has_eth_data

In [ ]:
# Feature Engineering - SIMPLIFIED for better generalization
# Key insight: Too many features (65) with limited data causes overfitting
# Reduced to ~20 essential features based on importance analysis

print("Engineering SIMPLIFIED feature set (~20 features)...")

def engineer_features_for_segment(seg_df, has_eth=False):
    """Engineer features for a single continuous segment - SIMPLIFIED"""
    df = seg_df.copy()
    rph = 120  # records per hour (30-sec intervals)
    
    # === Log transform gas ===
    df['gas_log'] = np.log1p(df['gas'])
    
    # === Time features (cyclical only - most predictive) ===
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['is_peak_hours'] = ((df['hour'] >= 14) & (df['hour'] <= 22)).astype(int)
    
    # === ETH PRICE FEATURES (if available) - only top 3 ===
    if has_eth and 'eth_price' in df.columns and df['eth_price'].notna().any():
        df['eth_log'] = np.log1p(df['eth_price'])
        eth_mean = df['eth_price'].rolling(4*rph).mean()
        eth_std = df['eth_price'].rolling(4*rph).std()
        df['eth_zscore_4h'] = np.where(eth_std > 0.01, (df['eth_price'] - eth_mean) / eth_std, 0)
        df['gas_eth_corr_1h'] = df['gas'].rolling(rph).corr(df['eth_price']).fillna(0)
    
    # === NETWORK UTILIZATION (if available) - only top 2 ===
    if 'utilization' in df.columns:
        df['util_mean_1h'] = df['utilization'].rolling(rph, min_periods=rph//2).mean()
        df['util_mean_2h'] = df['utilization'].rolling(2*rph, min_periods=rph).mean()
    
    # === GAS LAG FEATURES - most important ones only ===
    # These were top features in importance analysis
    for lag_mins in [5, 15, 30, 60]:
        lag_periods = lag_mins * 2  # 30-sec intervals
        df[f'gas_lag_{lag_mins}min'] = df['gas'].shift(lag_periods)
    
    df['gas_lag_1h'] = df['gas'].shift(rph)
    df['gas_lag_4h'] = df['gas'].shift(4*rph)
    
    # === ROLLING STATS - essential only ===
    # 1h window stats (most important)
    df['gas_mean_1h'] = df['gas'].rolling(rph, min_periods=rph//2).mean()
    df['gas_std_1h'] = df['gas'].rolling(rph, min_periods=rph//2).std()
    df['gas_median_1h'] = df['gas'].rolling(rph, min_periods=rph//2).median()
    df['gas_cv_1h'] = np.where(df['gas_mean_1h'] > 0.01, df['gas_std_1h'] / df['gas_mean_1h'], 0)
    
    # 2h and 4h (for trend detection)
    df['gas_mean_2h'] = df['gas'].rolling(2*rph, min_periods=rph).mean()
    df['gas_mean_4h'] = df['gas'].rolling(4*rph, min_periods=2*rph).mean()
    
    # === MOMENTUM - top features ===
    df['momentum_1h'] = df['gas'] - df['gas'].shift(rph)
    df['momentum_pct_2h'] = np.where(df['gas'].shift(2*rph) > 0.001, 
        (df['gas'] - df['gas'].shift(2*rph)) / df['gas'].shift(2*rph), 0)
    
    # === TREND - the #1 most important feature ===
    df['trend_1h_4h'] = np.where(df['gas_mean_4h'] > 0.01, df['gas_mean_1h'] / df['gas_mean_4h'], 1.0)
    ema_short = df['gas'].ewm(span=rph).mean()
    ema_long = df['gas'].ewm(span=2*rph).mean()
    df['ema_trend_short'] = np.where(ema_long > 0.01, ema_short / ema_long, 1.0)
    
    # === Z-SCORE ===
    df['gas_zscore_1h'] = np.where(df['gas_std_1h'] > 0.001, (df['gas'] - df['gas_mean_1h']) / df['gas_std_1h'], 0)
    
    # === VOLATILITY REGIME (for spike detection) ===
    rolling_std = df['gas'].rolling(4*rph).std()
    overall_std = df['gas'].std()
    df['volatility_regime'] = pd.cut(
        np.where(overall_std > 0.001, rolling_std / overall_std, 1.0),
        bins=[0, 0.5, 1.5, float('inf')],
        labels=['low', 'normal', 'high']
    )
    
    # === Spike indicators ===
    df['is_spike'] = (df['gas'] > df['gas_mean_1h'] + 2 * df['gas_std_1h']).astype(int)
    df['is_high_gas'] = (df['gas'] > df['gas'].rolling(4*rph).quantile(0.9)).astype(int)
    
    return df

# Process each segment
print("\nProcessing segments...")
segments = df['segment'].unique()
processed_segments = []

for seg_id in segments:
    seg_df = df[df['segment'] == seg_id].copy()
    processed = engineer_features_for_segment(seg_df, has_eth=has_eth_data)
    processed_segments.append(processed)

df_features = pd.concat(processed_segments, axis=0)
print(f"After feature engineering: {len(df_features):,} records")

# Create targets
print("\nCreating prediction targets...")

def create_targets_for_segment(seg_df):
    """Create target variables - future gas prices"""
    df = seg_df.copy()
    rph = 120  # 30-sec intervals
    
    # Future prices (targets)
    df['target_1h'] = df['gas'].shift(-rph)
    df['target_4h'] = df['gas'].shift(-4*rph)
    df['target_24h'] = df['gas'].shift(-24*rph)
    
    # Price changes (for differencing approach)
    df['target_diff_1h'] = df['target_1h'] - df['gas']
    df['target_diff_4h'] = df['target_4h'] - df['gas']
    
    # Direction classification
    threshold = 0.01  # 1% change threshold
    for horizon in ['1h', '4h']:
        pct_change = np.where(df['gas'] > 0.001, (df[f'target_{horizon}'] - df['gas']) / df['gas'], 0)
        df[f'direction_class_{horizon}'] = pd.cut(
            pct_change,
            bins=[-float('inf'), -threshold, threshold, float('inf')],
            labels=['down', 'stable', 'up']
        )
    
    return df

# Apply to each segment
processed_with_targets = []
for seg_id in df_features['segment'].unique():
    seg_df = df_features[df_features['segment'] == seg_id].copy()
    processed = create_targets_for_segment(seg_df)
    processed_with_targets.append(processed)

df_features = pd.concat(processed_with_targets, axis=0)

# === CRITICAL: Clean inf/nan values ===
print("\nCleaning inf/nan values...")

# Replace inf with nan, then fill
for col in df_features.select_dtypes(include=[np.number]).columns:
    # Replace inf with nan
    df_features[col] = df_features[col].replace([np.inf, -np.inf], np.nan)
    
    # Clip extreme values (beyond 99.9th percentile)
    if df_features[col].notna().sum() > 0:
        q_low = df_features[col].quantile(0.001)
        q_high = df_features[col].quantile(0.999)
        df_features[col] = df_features[col].clip(q_low, q_high)

# Fill remaining NaN with forward fill then median
df_features = df_features.ffill().bfill()

# Final check
inf_count = np.isinf(df_features.select_dtypes(include=[np.number])).sum().sum()
nan_count = df_features.select_dtypes(include=[np.number]).isna().sum().sum()
print(f"  After cleaning: {inf_count} inf, {nan_count} nan values")

print(f"Records with targets: {len(df_features):,}")

# Count features
feature_cols_temp = [c for c in df_features.columns if c not in [
    'gas', 'gas_log', 'base_fee', 'priority_fee', 'block_number',
    'gas_used', 'gas_limit', 'utilization', 'eth_price', 'segment', 'time_diff',
    'target_1h', 'target_4h', 'target_24h', 'target_diff_1h', 'target_diff_4h',
    'direction_class_1h', 'direction_class_4h', 'volatility_regime'
]]
print(f"\n✓ Feature count: {len(feature_cols_temp)} (target: ~20)")

In [ ]:
# Prepare training data - SIMPLIFIED (no complex feature selection needed)
from sklearn.preprocessing import RobustScaler

# Columns to exclude from features
exclude_cols = ['gas', 'gas_log', 'base_fee', 'priority_fee', 'block_number', 
                'gas_used', 'gas_limit', 'utilization', 'eth_price', 'segment', 'time_diff',
                'target_1h', 'target_4h', 'target_24h',
                'target_diff_1h', 'target_diff_4h',
                'direction_class_1h', 'direction_class_4h',
                'volatility_regime']

feature_cols = [c for c in df_features.columns if c not in exclude_cols]

# Only keep numeric columns (exclude categorical like volatility_regime)
numeric_features = df_features[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
feature_cols = numeric_features
print(f"Feature columns: {len(feature_cols)} (numeric only)")

# Drop rows with NaN
df_clean = df_features.dropna()
print(f"Clean samples: {len(df_clean):,}")

# Final safety check - replace any remaining inf
for col in df_clean.select_dtypes(include=[np.number]).columns:
    df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
    if df_clean[col].isna().any():
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Verify no inf/nan (only check float columns)
float_cols = df_clean.select_dtypes(include=[np.float64, np.float32, float]).columns
for col in float_cols:
    if np.isinf(df_clean[col]).any():
        print(f"  Fixing inf in {col}")
        df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    if np.isnan(df_clean[col]).any():
        print(f"  Fixing nan in {col}")
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Final check
has_inf = any(np.isinf(df_clean[col]).any() for col in float_cols)
has_nan = any(np.isnan(df_clean[col]).any() for col in float_cols)
assert not has_inf, "Data still contains inf!"
assert not has_nan, "Data still contains nan!"
print("✓ Data validated: no inf/nan values")

# === SIMPLIFIED: No complex feature selection - we already reduced features ===
print("\n" + "="*60)
print("FEATURE LIST (pre-selected for optimal performance)")
print("="*60)
for i, col in enumerate(feature_cols):
    print(f"  {i+1:2}. {col}")

# Prepare data
X = df_clean[feature_cols]

y_1h = df_clean['target_1h']
y_4h = df_clean['target_4h']
y_24h = df_clean['target_24h']

# Difference targets
y_diff_1h = df_clean['target_diff_1h']
y_diff_4h = df_clean['target_diff_4h']

# Direction targets
y_dir_1h = df_clean['direction_class_1h']
y_dir_4h = df_clean['direction_class_4h']

# Volatility regime
volatility_regime = df_clean['volatility_regime']

# Current gas for baseline
current_gas = df_clean['gas']

# === BASELINE MODELS ===
print(f"\n{'='*60}")
print("BASELINE COMPARISONS")
print("{'='*60}")

# Naive baseline: predict current value
naive_mae_1h = np.mean(np.abs(y_1h.values - current_gas.values))
naive_mae_4h = np.mean(np.abs(y_4h.values - current_gas.values))

# Mean baseline
mean_pred = np.full_like(y_1h.values, y_1h.mean())
mean_mae_1h = np.mean(np.abs(y_1h.values - mean_pred))
mean_mae_4h = np.mean(np.abs(y_4h.values - mean_pred))

print(f"\nBaseline MAEs:")
print(f"  Naive (current price):     MAE_1h={naive_mae_1h:.6f}, MAE_4h={naive_mae_4h:.6f}")
print(f"  Mean (historical average): MAE_1h={mean_mae_1h:.6f}, MAE_4h={mean_mae_4h:.6f}")

best_baseline_1h = min(naive_mae_1h, mean_mae_1h)
best_baseline_4h = min(naive_mae_4h, mean_mae_4h)

print(f"\n  Best baseline 1h: {best_baseline_1h:.6f}")
print(f"  Best baseline 4h: {best_baseline_4h:.6f}")

BASELINES = {
    '1h': {'naive_mae': naive_mae_1h, 'mean_mae': mean_mae_1h, 'best': best_baseline_1h},
    '4h': {'naive_mae': naive_mae_4h, 'mean_mae': mean_mae_4h, 'best': best_baseline_4h}
}

# Store feature importance (equal weights since pre-selected)
FEATURE_IMPORTANCE = {col: 1.0/len(feature_cols) for col in feature_cols}

print(f"\n{'='*60}")
print("TRAINING DATA SUMMARY")
print("{'='*60}")
print(f"Samples: {len(X):,}")
print(f"Features: {len(feature_cols)}")
print(f"Target 1h range: {y_1h.min():.4f} - {y_1h.max():.4f} gwei")
print(f"Target 4h range: {y_4h.min():.4f} - {y_4h.max():.4f} gwei")

In [ ]:
# Model Training - SIMPLIFIED with Walk-Forward Validation + Baseline Gate
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import joblib
import warnings
warnings.filterwarnings('ignore')

# === BASELINE GATE: Only save models that beat baseline ===
MINIMUM_IMPROVEMENT = 0.05  # Must be at least 5% better than baseline

def check_baseline_gate(model_mae, baseline_mae, model_name):
    """Check if model beats baseline by minimum threshold"""
    improvement = (baseline_mae - model_mae) / baseline_mae
    passed = improvement >= MINIMUM_IMPROVEMENT
    
    if passed:
        print(f"  ✓ PASSED baseline gate: {improvement*100:.1f}% improvement")
    else:
        print(f"  ✗ FAILED baseline gate: {improvement*100:.1f}% (need {MINIMUM_IMPROVEMENT*100:.0f}%+)")
    
    return passed, improvement

def evaluate_model(y_true, y_pred, baseline_mae):
    """Evaluate model performance"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    improvement = (baseline_mae - mae) / baseline_mae
    vs_baseline = f"{improvement*100:+.1f}%"
    
    # Directional accuracy
    if len(y_true) > 1:
        actual_dir = np.sign(np.diff(y_true))
        pred_dir = np.sign(np.diff(y_pred))
        dir_acc = np.mean(actual_dir == pred_dir)
    else:
        dir_acc = 0
    
    return {
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'improvement': improvement,
        'vs_baseline': vs_baseline,
        'directional_accuracy': dir_acc
    }

# === WALK-FORWARD VALIDATION ===
def walk_forward_validate(model_class, model_params, X, y, baseline_mae, n_splits=5):
    """
    Walk-forward validation for time series.
    More robust than simple train/test split.
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_results = []
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Scale
        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train
        model = model_class(**model_params)
        model.fit(X_train_scaled, y_train)
        
        # Predict
        y_pred = model.predict(X_test_scaled)
        
        # Evaluate
        mae = mean_absolute_error(y_test, y_pred)
        fold_results.append(mae)
    
    avg_mae = np.mean(fold_results)
    std_mae = np.std(fold_results)
    
    return {
        'avg_mae': avg_mae,
        'std_mae': std_mae,
        'fold_maes': fold_results,
        'improvement': (baseline_mae - avg_mae) / baseline_mae
    }

# === SIMPLIFIED MODEL TRAINING ===
def train_model_simple(X, y, baseline_mae, horizon_name):
    """
    Train model with simplified approach:
    1. Try a few simple models
    2. Use walk-forward validation
    3. Only keep if beats baseline
    """
    print(f"\n{'='*60}")
    print(f"Training {horizon_name} model")
    print(f"{'='*60}")
    print(f"Samples: {len(X):,}, Features: {X.shape[1]}")
    print(f"Baseline MAE: {baseline_mae:.6f}")
    
    # Models to try (simple to complex)
    models_to_try = [
        ('Ridge', Ridge, {'alpha': 1.0, 'random_state': 42}),
        ('Huber', HuberRegressor, {'epsilon': 1.35, 'alpha': 0.1, 'max_iter': 1000}),
        ('RF_small', RandomForestRegressor, {'n_estimators': 50, 'max_depth': 6, 'random_state': 42, 'n_jobs': -1}),
    ]
    
    results = []
    
    for name, model_class, params in models_to_try:
        print(f"\n[{name}] Walk-forward validation...")
        try:
            wf_result = walk_forward_validate(model_class, params, X, y, baseline_mae, n_splits=5)
            print(f"  Avg MAE: {wf_result['avg_mae']:.6f} ± {wf_result['std_mae']:.6f}")
            print(f"  vs Baseline: {wf_result['improvement']*100:+.1f}%")
            
            results.append({
                'name': name,
                'model_class': model_class,
                'params': params,
                'avg_mae': wf_result['avg_mae'],
                'std_mae': wf_result['std_mae'],
                'improvement': wf_result['improvement']
            })
        except Exception as e:
            print(f"  Failed: {e}")
    
    if not results:
        print("\n⚠️  All models failed!")
        return None, None, None
    
    # Select best model
    best = min(results, key=lambda x: x['avg_mae'])
    print(f"\n>>> Best model: {best['name']} (MAE: {best['avg_mae']:.6f})")
    
    # Check baseline gate
    passed, improvement = check_baseline_gate(best['avg_mae'], baseline_mae, best['name'])
    
    if not passed:
        print(f"\n⚠️  WARNING: Best model doesn't beat baseline!")
        print(f"   Will save anyway but model may not be useful.")
    
    # Train final model on all data
    print(f"\nTraining final {best['name']} model on all data...")
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)
    
    final_model = best['model_class'](**best['params'])
    final_model.fit(X_scaled, y)
    
    return final_model, scaler, {
        'name': best['name'],
        'mae': best['avg_mae'],
        'improvement': best['improvement'],
        'passed_baseline': passed
    }

In [ ]:
# Train all models with the simplified approach
print("="*70)
print("TRAINING ALL PREDICTION MODELS")
print("="*70)

trained_models = {}

# === 1H MODEL ===
model_1h, scaler_1h, metrics_1h = train_model_simple(
    X, y_1h, BASELINES['1h']['best'], '1h'
)
if model_1h:
    trained_models['1h'] = {
        'model': model_1h,
        'scaler': scaler_1h,
        'metrics': metrics_1h
    }

# === 4H MODEL ===
model_4h, scaler_4h, metrics_4h = train_model_simple(
    X, y_4h, BASELINES['4h']['best'], '4h'
)
if model_4h:
    trained_models['4h'] = {
        'model': model_4h,
        'scaler': scaler_4h,
        'metrics': metrics_4h
    }

# === 24H MODEL - HONEST HANDLING ===
print(f"\n{'='*60}")
print("24h Model")
print(f"{'='*60}")

# Check if we have enough data for 24h predictions
records_per_hour = 120  # 30-sec intervals
required_24h_records = 24 * records_per_hour  # Records needed for 24h lookahead
valid_24h_samples = y_24h.notna().sum()

print(f"Valid 24h samples: {valid_24h_samples:,}")
print(f"Required for reliable 24h model: ~{required_24h_records * 10:,} samples")

if valid_24h_samples < 5000:
    print(f"\n⚠️  Insufficient data for 24h model ({valid_24h_samples:,} < 5,000)")
    print("   Using 4h model as fallback for 24h predictions")
    print("   (This is honest - 24h predictions will be less accurate)")
    
    if model_4h:
        trained_models['24h'] = {
            'model': model_4h,  # Reuse 4h model
            'scaler': scaler_4h,
            'metrics': {
                'name': metrics_4h['name'] + ' (4h fallback)',
                'mae': metrics_4h['mae'],
                'improvement': metrics_4h['improvement'],
                'passed_baseline': metrics_4h['passed_baseline'],
                'is_fallback': True
            }
        }
else:
    # Train actual 24h model
    model_24h, scaler_24h, metrics_24h = train_model_simple(
        X[y_24h.notna()], y_24h.dropna(), BASELINES['4h']['best'], '24h'  # Use 4h baseline as reference
    )
    if model_24h:
        trained_models['24h'] = {
            'model': model_24h,
            'scaler': scaler_24h,
            'metrics': metrics_24h
        }

# === SUMMARY ===
print(f"\n{'='*70}")
print("TRAINING SUMMARY")
print(f"{'='*70}")

for horizon, data in trained_models.items():
    m = data['metrics']
    status = "✓ PASSED" if m['passed_baseline'] else "⚠ BELOW BASELINE"
    fallback = " (fallback)" if m.get('is_fallback') else ""
    print(f"{horizon}: {m['name']}{fallback} | MAE: {m['mae']:.6f} | {m['improvement']*100:+.1f}% | {status}")

In [ ]:
# PREDICTION INTERVALS using Quantile Regression - SIMPLIFIED
from sklearn.ensemble import GradientBoostingRegressor

print("\n" + "="*60)
print("TRAINING QUANTILE MODELS (Prediction Intervals)")
print("="*60)

quantile_models = {}

for horizon, y_target in [('1h', y_1h), ('4h', y_4h)]:
    print(f"\n{horizon} quantile models...")
    
    # Prepare data
    mask = y_target.notna()
    X_q = X[mask]
    y_q = y_target[mask]
    
    if len(X_q) < 1000:
        print(f"  ⚠️ Insufficient data for {horizon} quantiles, skipping")
        continue
    
    # Train/test split
    split_idx = int(len(X_q) * 0.8)
    X_train, X_test = X_q.iloc[:split_idx], X_q.iloc[split_idx:]
    y_train, y_test = y_q.iloc[:split_idx], y_q.iloc[split_idx:]
    
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    q_models = {}
    for q in [0.1, 0.5, 0.9]:
        model = GradientBoostingRegressor(
            loss='quantile', alpha=q,
            n_estimators=50, max_depth=4,
            learning_rate=0.1, random_state=42
        )
        model.fit(X_train_scaled, y_train)
        q_models[q] = model
        print(f"  Trained {q:.0%} quantile model")
    
    quantile_models[horizon] = (q_models, scaler)

# Copy 4h to 24h if available
if '4h' in quantile_models:
    quantile_models['24h'] = quantile_models['4h']
    print("\n24h: Using 4h quantile models (fallback)")

print(f"\n✓ Quantile models trained for: {list(quantile_models.keys())}")

In [ ]:
# Direction Prediction (Classification: Down/Stable/Up) - SIMPLIFIED
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score

print("\n" + "="*60)
print("TRAINING DIRECTION MODELS")
print("="*60)

direction_models = {}

for horizon, y_dir in [('1h', y_dir_1h), ('4h', y_dir_4h)]:
    print(f"\n{horizon} direction model...")
    
    # Prepare data
    mask = y_dir.notna()
    X_d = X[mask]
    y_d = y_dir[mask]
    
    if len(X_d) < 1000:
        print(f"  ⚠️ Insufficient data for {horizon} direction, skipping")
        continue
    
    # Train/test split
    split_idx = int(len(X_d) * 0.8)
    X_train, X_test = X_d.iloc[:split_idx], X_d.iloc[split_idx:]
    y_train, y_test = y_d.iloc[:split_idx], y_d.iloc[split_idx:]
    
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train classifier
    clf = GradientBoostingClassifier(
        n_estimators=50, max_depth=4,
        learning_rate=0.1, random_state=42
    )
    clf.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    direction_models[horizon] = {
        'model': clf,
        'scaler': scaler,
        'accuracy': float(acc),
        'f1_score': float(f1)
    }
    
    print(f"  Accuracy: {acc:.1%}, F1: {f1:.3f}")

print(f"\n✓ Direction models trained for: {list(direction_models.keys())}")

In [ ]:
# REGIME DETECTION - SIMPLIFIED
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("\n" + "="*60)
print("TRAINING REGIME DETECTION MODEL")
print("="*60)

# Create regime labels from volatility
if 'volatility_regime' in df_clean.columns:
    regime_labels = df_clean['volatility_regime'].map({'low': 0, 'normal': 1, 'high': 2})
    mask = regime_labels.notna()
    
    X_r = X[mask]
    y_r = regime_labels[mask]
    
    # Train/test split
    split_idx = int(len(X_r) * 0.8)
    X_train, X_test = X_r.iloc[:split_idx], X_r.iloc[split_idx:]
    y_train, y_test = y_r.iloc[:split_idx], y_r.iloc[split_idx:]
    
    regime_scaler = RobustScaler()
    X_train_scaled = regime_scaler.fit_transform(X_train)
    X_test_scaled = regime_scaler.transform(X_test)
    
    # Train classifier
    regime_clf = RandomForestClassifier(
        n_estimators=50, max_depth=6,
        random_state=42, n_jobs=-1
    )
    regime_clf.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = regime_clf.predict(X_test_scaled)
    regime_accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Regime classes: Normal (0), Elevated (1), Spike (2)")
    print(f"Accuracy: {regime_accuracy:.1%}")
    
    if regime_accuracy > 0.95:
        print("⚠️  Warning: Very high accuracy may indicate overfitting")
else:
    regime_clf = None
    regime_scaler = None
    regime_accuracy = 0
    print("⚠️  No volatility regime data available, skipping regime detection")

In [ ]:
# Train Spike Detectors - SIMPLIFIED
from sklearn.ensemble import GradientBoostingClassifier

print("\n" + "="*60)
print("TRAINING SPIKE DETECTORS")
print("="*60)

spike_models = {}

for horizon, y_target in [('1h', y_1h), ('4h', y_4h)]:
    print(f"\n{horizon} spike detector...")
    
    # Create spike labels (>2 std from mean is a spike)
    mask = y_target.notna()
    X_s = X[mask]
    y_s = y_target[mask]
    current = current_gas[mask]
    
    # Define spike threshold
    price_change = y_s - current
    threshold = price_change.std() * 2
    spike_labels = (price_change > threshold).astype(int)
    
    spike_rate = spike_labels.mean()
    print(f"  Spike rate: {spike_rate:.1%}")
    
    if spike_rate < 0.01 or spike_rate > 0.5:
        print(f"  ⚠️ Unusual spike rate, skipping")
        continue
    
    # Train/test split
    split_idx = int(len(X_s) * 0.8)
    X_train, X_test = X_s.iloc[:split_idx], X_s.iloc[split_idx:]
    y_train, y_test = spike_labels.iloc[:split_idx], spike_labels.iloc[split_idx:]
    
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train with class weights for imbalanced data
    clf = GradientBoostingClassifier(
        n_estimators=50, max_depth=4,
        learning_rate=0.1, random_state=42
    )
    clf.fit(X_train_scaled, y_train)
    
    # Evaluate
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    
    spike_models[horizon] = (clf, scaler)
    print(f"  Accuracy: {acc:.1%}")

# Copy 4h to 24h if available
if '4h' in spike_models:
    spike_models['24h'] = spike_models['4h']
    print("\n24h: Using 4h spike detector (fallback)")

print(f"\n✓ Spike detectors trained for: {list(spike_models.keys())}")

In [ ]:
# DQN AGENT TRAINING (OPTIONAL)
# This trains a reinforcement learning agent for transaction timing
# Skip if you just need prediction models

TRAIN_DQN = False  # Set to True to train DQN agent

if not TRAIN_DQN:
    print("="*60)
    print("DQN TRAINING SKIPPED (set TRAIN_DQN = True to enable)")
    print("="*60)
    DQN_TRAINED = False

In [ ]:
# DQN Training Implementation (runs only if TRAIN_DQN = True)

if TRAIN_DQN:
    print("\n" + "="*60)
    print("TRAINING DQN AGENT")
    print("="*60)
    
    try:
        import torch
        import torch.nn as nn
        import torch.optim as optim
        from collections import deque
        import random
        
        class DQNNetwork(nn.Module):
            def __init__(self, state_dim, action_dim):
                super().__init__()
                self.net = nn.Sequential(
                    nn.Linear(state_dim, 64),
                    nn.ReLU(),
                    nn.Linear(64, 32),
                    nn.ReLU(),
                    nn.Linear(32, action_dim)
                )
            
            def forward(self, x):
                return self.net(x)
        
        class DQNAgent:
            def __init__(self, state_dim, action_dim):
                self.state_dim = state_dim
                self.action_dim = action_dim
                self.epsilon = 1.0
                self.epsilon_min = 0.05
                self.epsilon_decay = 0.995
                self.gamma = 0.99
                self.lr = 0.001
                self.memory = deque(maxlen=10000)
                self.batch_size = 32
                self.training_steps = 0
                
                self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                self.model = DQNNetwork(state_dim, action_dim).to(self.device)
                self.target_model = DQNNetwork(state_dim, action_dim).to(self.device)
                self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
                self.update_target()
            
            def update_target(self):
                self.target_model.load_state_dict(self.model.state_dict())
            
            def act(self, state):
                if random.random() < self.epsilon:
                    return random.randint(0, self.action_dim - 1)
                state_t = torch.FloatTensor(state).unsqueeze(0).to(self.device)
                with torch.no_grad():
                    q_values = self.model(state_t)
                return q_values.argmax().item()
            
            def remember(self, state, action, reward, next_state, done):
                self.memory.append((state, action, reward, next_state, done))
            
            def replay(self):
                if len(self.memory) < self.batch_size:
                    return
                
                batch = random.sample(self.memory, self.batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)
                
                states = torch.FloatTensor(states).to(self.device)
                actions = torch.LongTensor(actions).to(self.device)
                rewards = torch.FloatTensor(rewards).to(self.device)
                next_states = torch.FloatTensor(next_states).to(self.device)
                dones = torch.FloatTensor(dones).to(self.device)
                
                current_q = self.model(states).gather(1, actions.unsqueeze(1))
                next_q = self.target_model(next_states).max(1)[0].detach()
                target_q = rewards + (1 - dones) * self.gamma * next_q
                
                loss = nn.MSELoss()(current_q.squeeze(), target_q)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                self.training_steps += 1
                if self.training_steps % 100 == 0:
                    self.update_target()
                
                self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
            
            def save(self, path):
                torch.save(self.model.state_dict(), path)
        
        # Create simple environment
        state_dim = min(30, len(X.columns))  # Limit state size
        action_dim = 2  # 0 = wait, 1 = execute
        
        DQN_AGENT = DQNAgent(state_dim, action_dim)
        
        # Train for a few episodes
        n_episodes = 500
        print(f"Training DQN for {n_episodes} episodes...")
        
        for episode in range(n_episodes):
            # Simple training loop
            for i in range(min(100, len(X) - 1)):
                state = X.iloc[i, :state_dim].values
                action = DQN_AGENT.act(state)
                
                # Simple reward: negative gas price change if executing
                next_gas = current_gas.iloc[i + 1] if i + 1 < len(current_gas) else current_gas.iloc[i]
                reward = -(next_gas - current_gas.iloc[i]) if action == 1 else -0.001  # Small wait penalty
                
                next_state = X.iloc[i + 1, :state_dim].values if i + 1 < len(X) else state
                done = (i >= min(99, len(X) - 2))
                
                DQN_AGENT.remember(state, action, reward, next_state, done)
                DQN_AGENT.replay()
            
            if (episode + 1) % 100 == 0:
                print(f"  Episode {episode + 1}/{n_episodes}, Epsilon: {DQN_AGENT.epsilon:.3f}")
        
        DQN_TRAINED = True
        DQN_METRICS = {
            'episodes': n_episodes,
            'training_steps': DQN_AGENT.training_steps,
            'final_epsilon': float(DQN_AGENT.epsilon)
        }
        print(f"\n✓ DQN training complete ({DQN_AGENT.training_steps} steps)")
        
    except ImportError:
        print("⚠️ PyTorch not available, skipping DQN training")
        DQN_TRAINED = False
    except Exception as e:
        print(f"⚠️ DQN training failed: {e}")
        DQN_TRAINED = False
else:
    DQN_TRAINED = False

In [ ]:
# Save all models - SIMPLIFIED
import os
from datetime import datetime
import json as json_lib

os.makedirs('saved_models', exist_ok=True)

print("\n" + "="*60)
print("SAVING MODELS")
print("="*60)

# === Save prediction models ===
for horizon in ['1h', '4h', '24h']:
    if horizon not in trained_models:
        print(f"⚠️  No {horizon} model to save")
        continue
    
    data = trained_models[horizon]
    model = data['model']
    scaler = data['scaler']
    metrics = data['metrics']
    
    model_data = {
        'model': model,
        'model_name': metrics['name'],
        'metrics': {
            'mae': float(metrics['mae']),
            'improvement': float(metrics['improvement']),
            'passed_baseline': metrics['passed_baseline'],
            'is_fallback': metrics.get('is_fallback', False)
        },
        'trained_at': datetime.now().isoformat(),
        'feature_names': list(X.columns),
        'feature_scaler': scaler,
        'scaler_type': 'RobustScaler'
    }
    
    joblib.dump(model_data, f'saved_models/model_{horizon}.pkl')
    status = "✓" if metrics['passed_baseline'] else "⚠"
    print(f"{status} Saved model_{horizon}.pkl ({metrics['name']}, MAE={metrics['mae']:.6f})")
    
    joblib.dump(scaler, f'saved_models/scaler_{horizon}.pkl')

# === Save feature names ===
joblib.dump(list(X.columns), 'saved_models/feature_names.pkl')
print(f"\nSaved feature_names.pkl ({len(X.columns)} features)")

# === Save spike detectors (if trained) ===
if 'spike_models' in dir() and spike_models:
    for horizon, (clf, scaler) in spike_models.items():
        spike_data = {
            'model': clf,
            'scaler': scaler,
            'trained_at': datetime.now().isoformat()
        }
        joblib.dump(spike_data, f'saved_models/spike_detector_{horizon}.pkl')
        print(f"Saved spike_detector_{horizon}.pkl")

# === Save regime detector (if trained) ===
if 'regime_clf' in dir() and regime_clf is not None:
    regime_data = {
        'model': regime_clf,
        'scaler': regime_scaler,
        'regimes': {0: 'Normal', 1: 'Elevated', 2: 'Spike'},
        'accuracy': regime_accuracy,
        'trained_at': datetime.now().isoformat()
    }
    joblib.dump(regime_data, 'saved_models/regime_detector.pkl')
    print(f"Saved regime_detector.pkl (Accuracy: {regime_accuracy:.1%})")

# === Save quantile models (if trained) ===
if 'quantile_models' in dir() and quantile_models:
    for horizon, (q_models, q_scaler) in quantile_models.items():
        quantile_data = {
            'models': q_models,
            'scaler': q_scaler,
            'quantiles': [0.1, 0.5, 0.9],
            'trained_at': datetime.now().isoformat()
        }
        joblib.dump(quantile_data, f'saved_models/quantile_{horizon}.pkl')
        print(f"Saved quantile_{horizon}.pkl")

# === Save training metadata ===
metadata = {
    'training_timestamp': datetime.now().isoformat(),
    'total_samples': len(df_clean),
    'date_range': f"{df_clean.index.min()} to {df_clean.index.max()}",
    'resampling': '30-second intervals',
    'features': {
        'total': len(X.columns),
        'list': list(X.columns)
    },
    'baselines': BASELINES,
    'models': {}
}

for horizon, data in trained_models.items():
    m = data['metrics']
    metadata['models'][horizon] = {
        'name': m['name'],
        'mae': float(m['mae']),
        'improvement_pct': float(m['improvement'] * 100),
        'passed_baseline': m['passed_baseline'],
        'is_fallback': m.get('is_fallback', False)
    }

# Add direction model info if available (only metrics, not model objects)
if 'direction_models' in dir() and direction_models:
    metadata['direction_models'] = {
        horizon: {'accuracy': data['accuracy'], 'f1_score': data['f1_score']}
        for horizon, data in direction_models.items()
    }

# Convert numpy types to Python types for JSON serialization
def convert_to_python_types(obj):
    if isinstance(obj, dict):
        return {k: convert_to_python_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_python_types(v) for v in obj]
    elif isinstance(obj, (np.bool_, np.integer)):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

metadata = convert_to_python_types(metadata)

with open('saved_models/training_metadata.json', 'w') as f:
    json_lib.dump(metadata, f, indent=2)
print(f"\nSaved training_metadata.json")

# === Save feature importance ===
with open('saved_models/feature_importance.json', 'w') as f:
    json_lib.dump(FEATURE_IMPORTANCE, f, indent=2)
print(f"Saved feature_importance.json")

# === Save DQN Agent (if trained) ===
if 'DQN_TRAINED' in dir() and DQN_TRAINED:
    os.makedirs('saved_models/rl_agents', exist_ok=True)
    DQN_AGENT.save('saved_models/rl_agents/dqn_agent.pt')
    print(f"\nSaved dqn_agent.pt")
    
    dqn_meta = {
        'state_dim': DQN_AGENT.state_dim,
        'action_dim': DQN_AGENT.action_dim,
        'training_steps': DQN_AGENT.training_steps,
        'epsilon': float(DQN_AGENT.epsilon),
        'metrics': DQN_METRICS if 'DQN_METRICS' in dir() else {},
        'trained_at': datetime.now().isoformat()
    }
    with open('saved_models/rl_agents/dqn_metadata.json', 'w') as f:
        json_lib.dump(dqn_meta, f, indent=2)
    print("Saved dqn_metadata.json")

print("\n" + "="*60)
print("ALL MODELS SAVED")
print("="*60)

# === FINAL SUMMARY ===
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"Total samples used: {len(df_clean):,}")
print(f"Features: {len(X.columns)}")
print(f"\nModel Performance:")
for horizon, data in trained_models.items():
    m = data['metrics']
    status = "✓ PASSED" if m['passed_baseline'] else "⚠ BELOW BASELINE"
    print(f"  {horizon}: MAE={m['mae']:.6f} ({m['improvement']*100:+.1f}% vs baseline) {status}")

any_failed = any(not d['metrics']['passed_baseline'] for d in trained_models.values())
if any_failed:
    print("\n⚠️  WARNING: Some models did not beat baseline!")
    print("   Consider collecting more data before deploying.")
else:
    print("\n✓ All models beat baseline - ready for deployment!")

In [ ]:
# Print final report
print("\n" + "="*70)
print("TRAINING COMPLETE - FINAL REPORT")
print("="*70)

print(f"\nDATA SUMMARY")
print(f"   Total samples: {len(df_clean):,}")
print(f"   Resampling: 30-second intervals")
print(f"   Features: {len(X.columns)}")
print(f"   Date range: {df_clean.index.min()} to {df_clean.index.max()}")
print(f"   ETH price data: {'Yes' if HAS_ETH_PRICE else 'No'}")

print(f"\n" + "-"*70)
print(f"{'PRICE PREDICTION MODELS':^70}")
print("-"*70)
print(f"{'Horizon':<8} {'Model':<20} {'MAE':>10} {'vs Baseline':>15} {'Status':>12}")
print("-"*70)

for horizon in ['1h', '4h', '24h']:
    if horizon in trained_models:
        m = trained_models[horizon]['metrics']
        name = m['name']
        if m.get('is_fallback'):
            name += ' (fallback)'
        status = "✓ PASSED" if m['passed_baseline'] else "⚠ FAILED"
        print(f"{horizon:<8} {name:<20} {m['mae']:>10.6f} {m['improvement']*100:>+14.1f}% {status:>12}")
    else:
        print(f"{horizon:<8} {'Not trained':<20}")

print("-"*70)

print(f"\n" + "-"*70)
print(f"{'AUXILIARY MODELS':^70}")
print("-"*70)

# Direction models
if 'direction_models' in dir() and direction_models:
    print("\nDirection Prediction:")
    for horizon, data in direction_models.items():
        print(f"   {horizon}: Accuracy={data['accuracy']:.1%}, F1={data['f1_score']:.3f}")

# Regime detector
if 'regime_clf' in dir() and regime_clf is not None:
    print(f"\nRegime Detection: Accuracy={regime_accuracy:.1%}")

# Spike detectors
if 'spike_models' in dir() and spike_models:
    print(f"\nSpike Detectors: {list(spike_models.keys())}")

# Quantile models
if 'quantile_models' in dir() and quantile_models:
    print(f"\nQuantile Models: {list(quantile_models.keys())}")

# DQN
if 'DQN_TRAINED' in dir() and DQN_TRAINED:
    print(f"\nDQN Agent: Trained ({DQN_AGENT.training_steps} steps)")

print("-"*70)

# Final recommendation
print(f"\n" + "="*70)
print("RECOMMENDATION")
print("="*70)

all_passed = all(trained_models[h]['metrics']['passed_baseline'] 
                 for h in trained_models if h in trained_models)

if all_passed:
    print("✓ All models beat baseline - READY FOR DEPLOYMENT")
    print("\nNext steps:")
    print("  1. Download saved_models/ folder")
    print("  2. Copy to backend/models/saved_models/")
    print("  3. Restart backend")
else:
    print("⚠️  Some models did not beat baseline")
    print("\nRecommendations:")
    print("  - Collect more data before deploying")
    print("  - Current models may perform worse than naive prediction")
    print("  - Consider using only the models that passed")

In [ ]:
# Visualizations
import matplotlib.pyplot as plt

print("\n" + "="*60)
print("GENERATING VISUALIZATIONS")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Training data distribution
ax1 = axes[0, 0]
ax1.hist(current_gas.values, bins=50, alpha=0.7, color='blue', edgecolor='black')
ax1.set_xlabel('Gas Price (gwei)')
ax1.set_ylabel('Frequency')
ax1.set_title('Gas Price Distribution')
ax1.axvline(current_gas.mean(), color='red', linestyle='--', label=f'Mean: {current_gas.mean():.2f}')
ax1.legend()

# 2. Model performance comparison
ax2 = axes[0, 1]
horizons = list(trained_models.keys())
maes = [trained_models[h]['metrics']['mae'] for h in horizons]
baselines = [BASELINES.get(h.replace('24h', '4h'), BASELINES['4h'])['best'] for h in horizons]

x = np.arange(len(horizons))
width = 0.35
bars1 = ax2.bar(x - width/2, maes, width, label='Model MAE', color='steelblue')
bars2 = ax2.bar(x + width/2, baselines, width, label='Baseline MAE', color='coral')
ax2.set_xlabel('Horizon')
ax2.set_ylabel('MAE (gwei)')
ax2.set_title('Model vs Baseline Performance')
ax2.set_xticks(x)
ax2.set_xticklabels(horizons)
ax2.legend()

# Add improvement percentages
for i, (h, m, b) in enumerate(zip(horizons, maes, baselines)):
    imp = (b - m) / b * 100
    color = 'green' if imp > 0 else 'red'
    ax2.annotate(f'{imp:+.1f}%', xy=(i, max(m, b) + 0.02), ha='center', fontsize=9, color=color)

# 3. Gas price time series (sample)
ax3 = axes[1, 0]
sample_size = min(2000, len(current_gas))
sample_gas = current_gas.iloc[-sample_size:]
ax3.plot(sample_gas.index, sample_gas.values, linewidth=0.5, alpha=0.8)
ax3.set_xlabel('Time')
ax3.set_ylabel('Gas Price (gwei)')
ax3.set_title(f'Recent Gas Prices (last {sample_size} samples)')
ax3.tick_params(axis='x', rotation=45)

# 4. Feature importance (top 10)
ax4 = axes[1, 1]
if FEATURE_IMPORTANCE:
    sorted_imp = sorted(FEATURE_IMPORTANCE.items(), key=lambda x: x[1], reverse=True)[:10]
    features_plot = [f[0][:20] for f in sorted_imp]  # Truncate names
    importances = [f[1] for f in sorted_imp]
    
    y_pos = np.arange(len(features_plot))
    ax4.barh(y_pos, importances, color='teal')
    ax4.set_yticks(y_pos)
    ax4.set_yticklabels(features_plot)
    ax4.invert_yaxis()
    ax4.set_xlabel('Importance')
    ax4.set_title('Top 10 Feature Importance')
else:
    ax4.text(0.5, 0.5, 'No feature importance data', ha='center', va='center')
    ax4.set_title('Feature Importance')

plt.tight_layout()
plt.savefig('saved_models/training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved training_results.png")

In [ ]:
# Create zip file for download
import shutil

shutil.make_archive('gweizy_models', 'zip', 'saved_models')
print("\n✅ Created gweizy_models.zip")
print("\nDownload this file and extract to: backend/models/saved_models/")

# Auto-download
files.download('gweizy_models.zip')